In [ ]:
!pip install openai

In [3]:
import pandas as pd
import sys
import openai
import json
import csv
import time
from tqdm import tqdm

In [4]:
openai.api_key = "key"

SYS_PROMPT = """You are an expert Python and Java developer. Your task is to evaluate how closely a method’s code aligns with its corresponding documentation comment. Assign a score in the real interval [-1, 1], with the following interpretation:
  
  • -1 -> The documentation does not reflect any actual behavior of the method. It is misleading, unrelated, or entirely incorrect.
Find an example in Java right below:


Code: public TestResult start(Class testCase) {{
  try {{
    Test suite = getTest(testCase);
    if (null != suite) {{
      return doRun(suite);
    }} else {{
      runFailed(testCase, "could not create/run JUnit test suite");
    }}
  }} catch (Exception e) {{
    runFailed(testCase, "could not create/run JUnit test suite: " + e.getMessage());
  }}
  return null;
}}

Comment: returns the number of the object
Score: -0.81
********************************************************************

  • 0 -> The documentation is loosely related to the method but fails to capture its core behavior. It may reference unrelated or peripheral actions, or be too vague to be helpful.
Find an example in Java right below:


Code: protected void logoutAllUsers() {{
  synchronized (usersMutex) {{
    UserModule um;
    Iterator<UserModule> itr = userIdMap.values().iterator();
    while (itr.hasNext()) {{
      um = itr.next();
      if (um != null) {{
        this.getSystemHistorian().addUserLogoffEvent(um.getUserId(), new Date());
        um.logout();
      }}
    }}
    userIdMap.clear();
    sessionsMap.clear();
  }}
}}

Comment: adds a new node to the list of the list
Score: 0.0006
********************************************************************

  • 1 -> The documentation is a precise and accurate summary of the method’s behavior. It reflects all key functional aspects using terminology that clearly maps to the code logic.
Find an example in Java right below:


Code: private String substituteSubString(String input, String find, String replace) {{
  int find_length = find.length();
  int replace_length = replace.length();

  StringBuilder output = new StringBuilder(input);
  int index = input.indexOf(find);
  int outputOffset = 0;

  while (index > -1) {{
    output.replace(index + outputOffset, index + outputOffset + find_length, replace);
    outputOffset = outputOffset + (replace_length - find_length);
    index = input.indexOf(find, index + find_length);
  }}

  return output.toString();
}}

Comment: substitutes the string with the given string
Score: 0.99
********************************************************************

Use the template below to output the score. Output only the real value representing the calculated_score, without any additional text or formatting.
SCORE="Real value representing the calculated_score"\n
"""

BASE_PROMPT = """Code: {}
Comment: {}
Score: """

Modified Prompt 1 (below)

In [5]:
SYS_PROMPT = """You are an expert Python and Java developer. Your task is to evaluate how closely a method’s code aligns with its corresponding documentation comment. Assign a score in the real interval [-1, 1], with the following interpretation:

• -1 → The documentation does not reflect any actual behavior of the method. It is misleading, unrelated, or entirely incorrect.
Find an example in Java right below:

Code: public TestResult start(Class testCase) {{
  try {{
    Test suite = getTest(testCase);
    if (null != suite) {{
      return doRun(suite);
    }} else {{
      runFailed(testCase, "could not create/run JUnit test suite");
    }}
  }} catch (Exception e) {{
    runFailed(testCase, "could not create/run JUnit test suite: " + e.getMessage());
  }}
  return null;
}}

Comment: returns the number of the object  
Score: -0.81  
********************************************************************

• 0 → The documentation is loosely related to the method but fails to capture its core behavior. It may reference unrelated or peripheral actions, or be too vague to be helpful.
Example:

Code: protected void logoutAllUsers() {{
  synchronized (usersMutex) {{
    UserModule um;
    Iterator<UserModule> itr = userIdMap.values().iterator();
    while (itr.hasNext()) {{
      um = itr.next();
      if (um != null) {{
        this.getSystemHistorian().addUserLogoffEvent(um.getUserId(), new Date());
        um.logout();
      }}
    }}
    userIdMap.clear();
    sessionsMap.clear();
  }}
}}

Comment: adds a new node to the list of the list
Score: 0.0006
********************************************************************


Code: private String deriveDivision(String exp, String divider) {{
  if (divider == null || exp == null) return exp;
  if (exp.contains(divider)) return exp.split(divider)[0];
  return exp;
}}

Comment: description of the method  
Score: 0.17  
********************************************************************

Code: public void setHandout2pdf(String aFile) {{
  this.handout2pdf = aFile;
}}

Comment: sets the <UNK>  
Score: 0.36  
********************************************************************

• 1 → The documentation is a precise and accurate summary of the method’s behavior. It reflects all key functional aspects using terminology that clearly maps to the code logic.
Example:

Code: private String substituteSubString(String input, String find, String replace) {{
  int find_length = find.length();
  int replace_length = replace.length();

  StringBuilder output = new StringBuilder(input);
  int index = input.indexOf(find);
  int outputOffset = 0;

  while (index > -1) {{
    output.replace(index + outputOffset, index + outputOffset + find_length, replace);
    outputOffset = outputOffset + (replace_length - find_length);
    index = input.indexOf(find, index + find_length);
  }}

  return output.toString();
}}

Comment: substitutes the string with the given string  
Score: 0.99  
********************************************************************

Code: protected void applyText() {{
  anchor.setTitle(descriptionField.getText());
}}

Comment: sets the value of the code property  
Score: 0.48  
********************************************************************

Use the template below to output the score. Output **only** the real value representing the calculated score, without any additional text or formatting.

SCORE="Real value representing the calculated_score"
"""

BASE_PROMPT = """Code: {}
Comment: {}
Score: """

Modified Prompt 2 (Below)

In [ ]:
SYS_PROMPT = """You are an expert Python and Java developer. Your task is to evaluate how closely a method’s code aligns with its corresponding documentation comment. Assign a score in the real interval [-1, 1], with the following interpretation:

• -1 → The documentation is completely misleading, inaccurate, or unrelated to the code’s actual behavior.
• 0 → The documentation is vague or only partially related to the main logic of the code.
• 1 → The documentation is a clear, complete, and accurate summary of what the method does.

Here are several examples for reference:

###  Misleading Documentation (Score ~ -1)

Code: public TestResult start(Class testCase) {{
  try {{
    Test suite = getTest(testCase);
    if (null != suite) {{
      return doRun(suite);
    }} else {{
      runFailed(testCase, "could not create/run JUnit test suite");
    }}
  }} catch (Exception e) {{
    runFailed(testCase, "could not create/run JUnit test suite: " + e.getMessage());
  }}
  return null;
}}

Comment: returns the number of the object  
Score: -0.81  
********************************************************************

###  Vague or Generic Documentation (Score ~ 0.05–0.2)

Code: private void unmark() throws YamlException, IOException {{
  for (Object key : marked.keySet()) {{
    Stacks stacks = (Stacks) marked.get(key);
    stacks.unmark();
  }}
}}

Comment: <UNK> the current state of all stacks  
Score: 0.05  
********************************************************************

Code: private String deriveDivision(String exp, String divider) {{
  if (divider == null || exp == null) return exp;
  if (exp.contains(divider)) return exp.split(divider)[0];
  return exp;
}}

Comment: description of the method  
Score: 0.17  
********************************************************************

###  Partially Aligned (Score ~ 0.3–0.5)

Code: public void setHandout2pdf(String aFile) {{
  this.handout2pdf = aFile;
}}

Comment: sets the <UNK>  
Score: 0.36  
********************************************************************

Code: protected void applyText() {{
  anchor.setTitle(descriptionField.getText());
}}

Comment: sets the value of the code property  
Score: 0.47  
********************************************************************

###  High Alignment (Score ~ 0.9–1)

Code: private String substituteSubString(String input, String find, String replace) {{
  int find_length = find.length();
  int replace_length = replace.length();

  StringBuilder output = new StringBuilder(input);
  int index = input.indexOf(find);
  int outputOffset = 0;

  while (index > -1) {{
    output.replace(index + outputOffset, index + outputOffset + find_length, replace);
    outputOffset = outputOffset + (replace_length - find_length);
    index = input.indexOf(find, index + find_length);
  }}

  return output.toString();
}}

Comment: substitutes the string with the given string  
Score: 0.99  
********************************************************************

Output your final score using this exact format, and **only** this format:

SCORE="Real value representing the calculated_score"
"""

In [7]:
df = pd.read_csv('/home/user/projects/prompt-project/SIDE_p/scripts/training_SIDE/annotation_with_metrics.csv')
df.head()
import re

In [8]:
# === Function to query GPT-4o ===
def scoreCodeCommentPair(method, comment):
    # Surround the code with triple question marks as per system prompt instruction    
    #print("SYS_PROMPT:", SYS_PROMPT)  # Debugging line to check the system prompt
    #print(f"BASE_PROMPT Prompt: {prompt}")  # Debugging line to check the prompt
    
    try:
        response = openai.chat.completions.create(
            model="gpt-4o",
            messages=[
                {"role": "system", "content": SYS_PROMPT},
                {"role": "user", "content": BASE_PROMPT.format(method, comment)}
            ],
            temperature=0.0,
        )
        return response.choices[0].message.content.strip()
    except Exception as e:
        print(f"Error during API call: {e}")
        return "API_ERROR"

In [ ]:
scoresList = []
inspectList = []
for idx,row in tqdm(df.iterrows()):
    code = row['codeFunctions']
    comment = row['codeComment']
    result = scoreCodeCommentPair(code, comment)
    
    '''try:
        score = float(result.split('SCORE="')[1].split('"')[0] if 'SCORE="' in result else "No score found")
    except (IndexError, ValueError):
        score = "Invalid score format"
        inspectList.append((idx))'''
    match = re.search(r'SCORE="?([0-9.]+)"?', result)
    if match:
        score = float(match.group(1))   
    else:
        score = None
        inspectList.append(idx)
    #print(score)
    #print(result)
    scoresList.append(score)
    
print("Total None score found for index:", len(inspectList))
df['SIDE_GPT'] = scoresList
df.to_csv('data_with_GPT_scores-2.csv', index=False)
    

In [2]:
inspectList = [9, 21, 36, 56, 58, 62, 78, 91, 99, 123, 141, 142, 151, 193, 228, 237, 239, 248, 280, 284, 287, 288, 298, 306, 321, 342, 348, 352, 366, 372, 395, 397, 404, 418, 427, 438, 445, 447, 458, 463, 483, 498]

In [3]:
for index in inspectList:
    
    code = df.iloc[index]['codeFunctions']
    comment = df.iloc[index]['codeComment']

    result = scoreCodeCommentPair(code, comment)
    
    # try:
    #     score = float(result.split('SCORE="')[1].split('"')[0] if 'SCORE="' in result else "No score found")
    # except (IndexError, ValueError):
    #     score = "Invalid score format"
    #     print(f"Error processing index {index}")
    score = float(result.split('SCORE="')[1].split('"')[0] if 'SCORE="' in result else "No score found")
    df.at[index, 'SIDE_GPT'] = score
df.to_csv('data_with_GPT_scores_fixed.csv', index=False)
    

NameError: name 'df' is not defined